In [41]:
#import libraries

import plotly.graph_objects as go
import plotly.express as px
import os
import pandas as pd
import hvplot.pandas
import hvplot
import requests
import numpy as np
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
from datetime import datetime
from itertools import islice
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [42]:
#load .env file

load_dotenv()

True

In [43]:
# Set Alpaca API key and secret

alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [44]:
# Initiate REST API

api = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

In [45]:
# Request stock symbol from user. This code will work will all available tickers on alpacas.
# "META" ticker has been temporarily hard coded into the system for ease of use

# stock_symbol = input("Enter the stock symbol you want to analyze (e.g., AAPL): ")

stock_symbol = "META"

In [46]:
# Establish what time frame increment we will be looking at
# currently we are working with the daily chart, but the theory would be to 
# allow the user to identify what time scale they would like to trade on
# shorter time frame allows for "day trades" larger time scales are geared
# more towards longer term investments and swing trading.

time_frame = "1Day"

In [47]:
# Get user input for the start date in datetime format
# the start date has been hard coded for ease of programming and troubleshooting our code

# start_date_str = input("Enter the start date (YYYY-MM-DD): ")
# start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
start_date = pd.Timestamp("2016-01-01", tz="America/New_York").isoformat()

In [48]:
# Get user input for the end date in datetime format
# the end date has been hard coded for ease of programming and troubleshooting our code

# end_date_str = input("Enter the end date (YYYY-MM-DD): ")
# end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
end_date = pd.Timestamp("2020-01-01", tz="America/New_York").isoformat()

In [49]:
# trading_days = end_date - start_date

# print(f"The timeframe between {start_date_str} and {end_date_str} is {trading_days.days} days.")

In [50]:
# Calls the stock data from the period of time desired in the desired time increment

stock_data = api.get_bars(
    stock_symbol, 
    time_frame, 
    start = start_date, 
    end = end_date
).df

In [51]:
# Displays the information pulled for working through code

stock_data.info()
display(stock_data.head())
display(stock_data.tail())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1006 entries, 2016-01-04 05:00:00+00:00 to 2019-12-31 05:00:00+00:00
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   close        1006 non-null   float64
 1   high         1006 non-null   float64
 2   low          1006 non-null   float64
 3   trade_count  1006 non-null   int64  
 4   open         1006 non-null   float64
 5   volume       1006 non-null   int64  
 6   vwap         1006 non-null   float64
dtypes: float64(5), int64(2)
memory usage: 62.9 KB


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2016-01-04 05:00:00+00:00,102.22,102.2400,99.7500,234429,101.95,40592464,101.342317
2016-01-05 05:00:00+00:00,102.73,103.7099,101.6650,143230,102.89,24021945,102.713971
2016-01-06 05:00:00+00:00,102.97,103.7700,100.9000,152003,101.13,25678428,102.659946
2016-01-07 05:00:00+00:00,97.92,101.4300,97.3000,271754,100.50,46469979,99.346563
2016-01-08 05:00:00+00:00,97.33,100.5000,97.0311,204105,99.88,36533649,98.901944


,close,high,low,trade_count,open,volume,vwap
timestamp,,,,,,,
2019-12-24 05:00:00+00:00,205.12,206.79,205.0000,38407,206.3000,6455754,205.557384
2019-12-26 05:00:00+00:00,207.79,207.82,205.3100,67748,205.5665,10573037,207.205749
2019-12-27 05:00:00+00:00,208.10,208.93,206.5875,86780,208.6697,11622930,208.011886
2019-12-30 05:00:00+00:00,204.41,207.90,203.9001,91911,207.8600,11776302,204.992696
2019-12-31 05:00:00+00:00,205.25,205.56,203.6001,70904,203.9950,10409305,204.798742


In [52]:
# Reset index and display the first 5
# this also prepares for use with plotly library/visualization

#stock_data_plot = stock_data.reset_index()

# clean up datafram by removing vwap, volume, trade count columns
# Drop columns which aren't needed(trade_count, volume, vwap) and display the results

stock_data = stock_data.drop(columns=['trade_count','volume','vwap'])
stock_data_plot = stock_data.reset_index()

stock_data.head()

,close,high,low,open
timestamp,,,,
2016-01-04 05:00:00+00:00,102.22,102.2400,99.7500,101.95
2016-01-05 05:00:00+00:00,102.73,103.7099,101.6650,102.89
2016-01-06 05:00:00+00:00,102.97,103.7700,100.9000,101.13
2016-01-07 05:00:00+00:00,97.92,101.4300,97.3000,100.50
2016-01-08 05:00:00+00:00,97.33,100.5000,97.0311,99.88


In [53]:
# Assign plot values to plotly

fig = go.Candlestick(x=stock_data_plot['timestamp'],
                open=stock_data_plot['open'],
                high=stock_data_plot['high'],
                low=stock_data_plot['low'],
                close=stock_data_plot['close'])


cand = go.Figure(data=[fig])

In [54]:
# Customize the figure (optional)

cand.update_layout(
    width=1200, height=800,
    title="META, OCT - DEC",
    yaxis_title='META Stock'
)


In [55]:
# Add new data columns to DF for body size and candle size
# body size and direction is value of open - close. 
# candle size is the absolute value of low - high

stock_data['body_size'] = stock_data['close'] - stock_data['open']
stock_data['candle_size'] = stock_data['high'] - stock_data['low']

# Display columns with added columns

stock_data.head()

,close,high,low,open,body_size,candle_size
timestamp,,,,,,
2016-01-04 05:00:00+00:00,102.22,102.2400,99.7500,101.95,0.27,2.4900
2016-01-05 05:00:00+00:00,102.73,103.7099,101.6650,102.89,-0.16,2.0449
2016-01-06 05:00:00+00:00,102.97,103.7700,100.9000,101.13,1.84,2.8700
2016-01-07 05:00:00+00:00,97.92,101.4300,97.3000,100.50,-2.58,4.1300
2016-01-08 05:00:00+00:00,97.33,100.5000,97.0311,99.88,-2.55,3.4689


In [66]:
# initialize variables to be used in calculations
# identify the max body size and minimum candle size within the dataframe

max_body_size = stock_data["body_size"].max()
min_candle_size = stock_data["candle_size"].min()

# keeps track of the count which would initiate a buy or sell
# current count is initialized at 0 as the count is calculated based off of historical data

current_count = 0

# the threshold that would initiate a buy and or sell

threshold = 2

# adds a documenting column to the dataframe and assigns a flat value to all rows

stock_data["card_count"] = current_count

# identifies the number of rows to be used to break the for loop

number_of_rows = len(stock_data)

# print information up to date to identify data progress

# print(max_body_size)
# print(min_candle_size)
# print(previous_candle)
# print(stock_data.iloc[0][4])
# print(number_of_rows)
# stock_data.head()

In [67]:
# ...::: working code :::...
# initialize variables and dataframe for tracking buys, sells, and profits

initial_investment = 10000

# initialize strategy tracking variables and add to stock_data df

stock_data["capital"] = initial_investment
stock_data["number_of_shares"] = 0
stock_data["investment"] = 0
stock_data["profits"] = 0

# variables that will be used to calculate profits/losses from buys/sells

capital = initial_investment
number_of_shares = 0
investment = 0
profits = 0

stock_data.head()

,close,high,low,open,body_size,candle_size,card_count,capital,number_of_shares,investment,profits,comp_profits,total_profits,port_size
timestamp,,,,,,,,,,,,,,
2016-01-04 05:00:00+00:00,102.22,102.2400,99.7500,101.95,0.27,2.4900,0,10000,0,0,0,-84.66,0.0,10000.0
2016-01-05 05:00:00+00:00,102.73,103.7099,101.6650,102.89,-0.16,2.0449,0,10000,0,0,0,-35.19,0.0,10000.0
2016-01-06 05:00:00+00:00,102.97,103.7700,100.9000,101.13,1.84,2.8700,0,10000,0,0,0,-11.91,0.0,10000.0
2016-01-07 05:00:00+00:00,97.92,101.4300,97.3000,100.50,-2.58,4.1300,0,10000,0,0,0,-501.76,0.0,10000.0
2016-01-08 05:00:00+00:00,97.33,100.5000,97.0311,99.88,-2.55,3.4689,0,10000,0,0,0,-558.99,0.0,10000.0


In [68]:
# initialize variables and assign values based off of first row for calculations

previous_close = stock_data.iloc[0][0]
previous_high = stock_data.iloc[0][1]
previous_low = stock_data.iloc[0][2]
previous_open = stock_data.iloc[0][3]
previous_body_size = stock_data.iloc[0][4]
previous_candle_size = stock_data.iloc[0][5]

# begin for loop to start calculating each row's data to assign a +1 or -1
# to our count and add adjust our current/previous counts accordingly

for index, row in islice(stock_data.iterrows(), 1, None):

# create an if statement to make sure the for loop finishes at the end of the data appropriately  
    
    if index == (number_of_rows):
        break
    
# assign current day numbers to work with conditional statements

    current_close = row[0]
    current_high = row[1]
    current_low = row[2]
    current_open = row[3]
    current_body_size = row[4]
    current_candle_size = row[5]
    # capital = row[7]
    # number_of_shares = row[8]
    # investment = row[9]
    # profits = row[10]

   # begin of conditional statements. this portion identifies what values to assign
   # to each individal row's data and to determine whether a threshold of "sell" or "buy"
   # is reached. this portion can be adjusted/added to in order to take into consideration
   # additional bar patterns

    # bullish hammer
    if (current_body_size > 0) and (current_body_size*2) < (current_candle_size):
        current_count = current_count + 1
    # bearish hammer
    elif (current_body_size < 0) and ((np.absolute(current_body_size)*2) < (current_candle_size)):
        current_count = current_count - 1
    # bullish engulfing candle
    elif (current_body_size > 0) and (previous_body_size < 1) and (current_body_size > previous_body_size):
        current_count = current_count + 1
    # bearish engulfing candle
    elif (current_body_size < 0) and (previous_body_size > 1) and (np.absolute(current_body_size) > (previous_body_size)):
        current_count = current_count - 1
    # bullish doji candle
    #elif (previous_body_size < 0) and (np.absolute(current_body_size) < (current_close * max_body_size)):
    #    current_count = current_count + 1

    # contains the current count actual value to the desired "buy/sell" threshold

    if current_count > threshold:
        current_count = threshold
    elif current_count < -(threshold):
        current_count = -(threshold)

    # conditional statements to identify when to purchase or sell a stock

    if (current_count == threshold) and (number_of_shares == 0):
        number_of_shares = np.floor(capital / current_close)
        investment = number_of_shares * current_close
        capital = capital - investment
    elif (current_count == -threshold) and (number_of_shares != 0):
        profits = (number_of_shares * current_close) - investment
        capital = capital + investment + profits
        investment = 0
        number_of_shares = 0
    else:
        profits = (number_of_shares * current_close) - investment

        #stock_data.at[index,"investment"] = number_of_shares * current_close
        #profits = row[9] - investment
        #stock_data.at[index,"profits"] = profits

    stock_data.at[index,"number_of_shares"] = number_of_shares
    stock_data.at[index,"investment"] = investment
    stock_data.at[index,"profits"] = profits
    stock_data.at[index,"capital"] = capital
    # set variables to prepare to analyze the following row

    previous_close = row[0]
    previous_high = row[1]
    previous_low = row[2]
    previous_open = row[3]
    previous_body_size = row[4]
    previous_candle_size = row[5]

    # keep track of the current count within the dataframe. assign that
    # to the current working row
    
    stock_data.at[index,"card_count"] = current_count

if (number_of_shares == 0):
    total_profit = capital
elif (number_of_shares != 0):
    total_profit = (capital + investment + profits) - initial_investment

total_profit = round(total_profit,2)
percent_profit = (total_profit/initial_investment)*100

print(f"total profits for your autotrader are: ${total_profit} a {percent_profit}%")
    
display(stock_data.head(40))
display(stock_data.tail(40))





total profits for your autotrader are: $5132.92 a 51.3292%


,close,high,low,open,body_size,candle_size,card_count,capital,number_of_shares,investment,profits,comp_profits,total_profits,port_size
timestamp,,,,,,,,,,,,,,
2016-01-04 05:00:00+00:00,102.22,102.2400,99.7500,101.95,0.27,2.4900,0,10000.00,0,0.00,0.00,-84.66,0.0,10000.0
2016-01-05 05:00:00+00:00,102.73,103.7099,101.6650,102.89,-0.16,2.0449,-1,10000.00,0,0.00,0.00,-35.19,0.0,10000.0
2016-01-06 05:00:00+00:00,102.97,103.7700,100.9000,101.13,1.84,2.8700,0,10000.00,0,0.00,0.00,-11.91,0.0,10000.0
2016-01-07 05:00:00+00:00,97.92,101.4300,97.3000,100.50,-2.58,4.1300,-1,10000.00,0,0.00,0.00,-501.76,0.0,10000.0
2016-01-08 05:00:00+00:00,97.33,100.5000,97.0311,99.88,-2.55,3.4689,-1,10000.00,0,0.00,0.00,-558.99,0.0,10000.0
2016-01-11 05:00:00+00:00,97.51,98.6000,95.3900,97.91,-0.40,3.2100,-2,10000.00,0,0.00,0.00,-541.53,0.0,10000.0
2016-01-12 05:00:00+00:00,99.37,99.9600,97.5500,99.00,0.37,2.4100,-1,10000.00,0,0.00,0.00,-361.11,0.0,10000.0
2016-01-13 05:00:00+00:00,95.44,100.5800,95.2100,100.58,-5.14,5.3700,-1,10000.00,0,0.00,0.00,-742.32,0.0,10000.0
2016-01-14 05:00:00+00:00,98.37,98.8700,92.4500,95.85,2.52,6.4200,0,10000.00,0,0.00,0.00,-458.11,0.0,10000.0


,close,high,low,open,body_size,candle_size,card_count,capital,number_of_shares,investment,profits,comp_profits,total_profits,port_size
timestamp,,,,,,,,,,,,,,
2019-11-04 05:00:00+00:00,194.72,197.3672,193.8119,194.5500,0.1700,3.5553,2,149.67,73,13172.85,1041.71,8887.84,7345.41,17345.41
2019-11-05 05:00:00+00:00,194.32,195.7500,193.6000,195.3700,-1.0500,2.1500,1,149.67,73,13172.85,1012.51,8849.04,7309.81,17309.81
2019-11-06 05:00:00+00:00,191.55,194.3700,191.3500,194.0300,-2.4800,3.0200,1,149.67,73,13172.85,810.30,8580.35,7063.28,17063.28
2019-11-07 05:00:00+00:00,190.42,193.4400,189.4700,191.9100,-1.4900,3.9700,0,149.67,73,13172.85,727.81,8470.74,6962.71,16962.71
2019-11-08 05:00:00+00:00,190.84,192.3400,189.7000,190.0000,0.8400,2.6400,1,149.67,73,13172.85,758.47,8511.48,7000.09,17000.09
2019-11-11 05:00:00+00:00,189.61,190.0800,188.5400,189.8100,-0.2000,1.5400,0,149.67,73,13172.85,668.68,8392.17,6890.62,16890.62
2019-11-12 05:00:00+00:00,194.47,195.0600,189.7400,190.0000,4.4700,5.3200,1,149.67,73,13172.85,1023.46,8863.59,7323.16,17323.16
2019-11-13 05:00:00+00:00,193.19,195.7000,192.7400,194.7000,-1.5100,2.9600,1,149.67,73,13172.85,930.02,8739.43,7209.24,17209.24
2019-11-14 05:00:00+00:00,193.15,194.0300,191.4500,192.9300,0.2200,2.5800,2,149.67,73,13172.85,927.10,8735.55,7205.68,17205.68


In [69]:
comp_number_of_shares = np.floor(initial_investment / stock_data.iloc[0][0])
stock_data["comp_profits"] = (comp_number_of_shares * stock_data["close"]) - initial_investment
stock_data["total_profits"] = (stock_data["capital"] + stock_data["investment"] + stock_data["profits"]) - initial_investment
print(comp_number_of_shares)
display(stock_data.head(40))
display(stock_data.tail(40))


97.0


,close,high,low,open,body_size,candle_size,card_count,capital,number_of_shares,investment,profits,comp_profits,total_profits,port_size
timestamp,,,,,,,,,,,,,,
2016-01-04 05:00:00+00:00,102.22,102.2400,99.7500,101.95,0.27,2.4900,0,10000.00,0,0.00,0.00,-84.66,0.00,10000.0
2016-01-05 05:00:00+00:00,102.73,103.7099,101.6650,102.89,-0.16,2.0449,-1,10000.00,0,0.00,0.00,-35.19,0.00,10000.0
2016-01-06 05:00:00+00:00,102.97,103.7700,100.9000,101.13,1.84,2.8700,0,10000.00,0,0.00,0.00,-11.91,0.00,10000.0
2016-01-07 05:00:00+00:00,97.92,101.4300,97.3000,100.50,-2.58,4.1300,-1,10000.00,0,0.00,0.00,-501.76,0.00,10000.0
2016-01-08 05:00:00+00:00,97.33,100.5000,97.0311,99.88,-2.55,3.4689,-1,10000.00,0,0.00,0.00,-558.99,0.00,10000.0
2016-01-11 05:00:00+00:00,97.51,98.6000,95.3900,97.91,-0.40,3.2100,-2,10000.00,0,0.00,0.00,-541.53,0.00,10000.0
2016-01-12 05:00:00+00:00,99.37,99.9600,97.5500,99.00,0.37,2.4100,-1,10000.00,0,0.00,0.00,-361.11,0.00,10000.0
2016-01-13 05:00:00+00:00,95.44,100.5800,95.2100,100.58,-5.14,5.3700,-1,10000.00,0,0.00,0.00,-742.32,0.00,10000.0
2016-01-14 05:00:00+00:00,98.37,98.8700,92.4500,95.85,2.52,6.4200,0,10000.00,0,0.00,0.00,-458.11,0.00,10000.0


,close,high,low,open,body_size,candle_size,card_count,capital,number_of_shares,investment,profits,comp_profits,total_profits,port_size
timestamp,,,,,,,,,,,,,,
2019-11-04 05:00:00+00:00,194.72,197.3672,193.8119,194.5500,0.1700,3.5553,2,149.67,73,13172.85,1041.71,8887.84,4364.23,17345.41
2019-11-05 05:00:00+00:00,194.32,195.7500,193.6000,195.3700,-1.0500,2.1500,1,149.67,73,13172.85,1012.51,8849.04,4335.03,17309.81
2019-11-06 05:00:00+00:00,191.55,194.3700,191.3500,194.0300,-2.4800,3.0200,1,149.67,73,13172.85,810.30,8580.35,4132.82,17063.28
2019-11-07 05:00:00+00:00,190.42,193.4400,189.4700,191.9100,-1.4900,3.9700,0,149.67,73,13172.85,727.81,8470.74,4050.33,16962.71
2019-11-08 05:00:00+00:00,190.84,192.3400,189.7000,190.0000,0.8400,2.6400,1,149.67,73,13172.85,758.47,8511.48,4080.99,17000.09
2019-11-11 05:00:00+00:00,189.61,190.0800,188.5400,189.8100,-0.2000,1.5400,0,149.67,73,13172.85,668.68,8392.17,3991.20,16890.62
2019-11-12 05:00:00+00:00,194.47,195.0600,189.7400,190.0000,4.4700,5.3200,1,149.67,73,13172.85,1023.46,8863.59,4345.98,17323.16
2019-11-13 05:00:00+00:00,193.19,195.7000,192.7400,194.7000,-1.5100,2.9600,1,149.67,73,13172.85,930.02,8739.43,4252.54,17209.24
2019-11-14 05:00:00+00:00,193.15,194.0300,191.4500,192.9300,0.2200,2.5800,2,149.67,73,13172.85,927.10,8735.55,4249.62,17205.68


In [70]:
# Plot the Comparison between Total Profits and Comparative Profits

stock_data.hvplot.line(x="timestamp", y=('total_profits', 'comp_profits'), title= 'RELATIONSHIP BETWEEN TOTAL PROFITS AND COMPARATIVE PROFITS')


:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [71]:
# Plot the Number of Shares in Line Graph

stock_data.hvplot.line(x='timestamp', y='number_of_shares', title='SHARES GROWTH OVER THE YEARS')

:Curve   [timestamp]   (number_of_shares)

In [72]:
# Analysis results dataframe and display dataframe

stock_data['port_size'] = stock_data['capital'] + stock_data['profits'] + stock_data['investment']

display(stock_data.head())
display(stock_data.tail())

,close,high,low,open,body_size,candle_size,card_count,capital,number_of_shares,investment,profits,comp_profits,total_profits,port_size
timestamp,,,,,,,,,,,,,,
2016-01-04 05:00:00+00:00,102.22,102.2400,99.7500,101.95,0.27,2.4900,0,10000.0,0,0.0,0.0,-84.66,0.0,10000.0
2016-01-05 05:00:00+00:00,102.73,103.7099,101.6650,102.89,-0.16,2.0449,-1,10000.0,0,0.0,0.0,-35.19,0.0,10000.0
2016-01-06 05:00:00+00:00,102.97,103.7700,100.9000,101.13,1.84,2.8700,0,10000.0,0,0.0,0.0,-11.91,0.0,10000.0
2016-01-07 05:00:00+00:00,97.92,101.4300,97.3000,100.50,-2.58,4.1300,-1,10000.0,0,0.0,0.0,-501.76,0.0,10000.0
2016-01-08 05:00:00+00:00,97.33,100.5000,97.0311,99.88,-2.55,3.4689,-1,10000.0,0,0.0,0.0,-558.99,0.0,10000.0


,close,high,low,open,body_size,candle_size,card_count,capital,number_of_shares,investment,profits,comp_profits,total_profits,port_size
timestamp,,,,,,,,,,,,,,
2019-12-24 05:00:00+00:00,205.12,206.79,205.0000,206.3000,-1.1800,1.7900,0,149.67,73,13172.85,1800.91,9896.64,5123.43,15123.43
2019-12-26 05:00:00+00:00,207.79,207.82,205.3100,205.5665,2.2235,2.5100,1,149.67,73,13172.85,1995.82,10155.63,5318.34,15318.34
2019-12-27 05:00:00+00:00,208.10,208.93,206.5875,208.6697,-0.5697,2.3425,0,149.67,73,13172.85,2018.45,10185.70,5340.97,15340.97
2019-12-30 05:00:00+00:00,204.41,207.90,203.9001,207.8600,-3.4500,3.9999,0,149.67,73,13172.85,1749.08,9827.77,5071.60,15071.60
2019-12-31 05:00:00+00:00,205.25,205.56,203.6001,203.9950,1.2550,1.9599,1,149.67,73,13172.85,1810.40,9909.25,5132.92,15132.92


In [73]:
#Plot Line Graph for Portfolio Size

stock_data.hvplot.line(x='timestamp', y='port_size', title='PORTFOLIO SIZE OVER THE YEARS')

:Curve   [timestamp]   (port_size)

In [74]:
# Plot a Line Graph for Card Count

stock_data.hvplot.line(x='timestamp', y='card_count', title='CARD COUNT OVER THE YEARS')

:Curve   [timestamp]   (card_count)

In [75]:
# Plot a Scatter Graph for Card Count

stock_data.hvplot.scatter(x='timestamp', y='card_count', by='port_size', legend= 'bottom', height=400, width=800, title= 'RELATIONSHIP BETWEEN CARD COUNT AND PORT SIZE')

:NdOverlay   [port_size]
   :Scatter   [timestamp]   (card_count)